# Working with pre-extracted embeddings

When you're testing a Few-Shot Learning models, you are going to solve hundreds of randomly sampled few-shot tasks. In doing so, you are more than likely to process the same images several times. This means that these images will go through your backbone several times, which is a waste in time and energy. Indeed, most Few-Shot Learning methods nowadays make use of a **frozen backbone**: the logic of these methods is at the feature level. Therefore, you can extract the features of your images once and for all, and then use these features to solve your few-shot tasks.

All the necessary tools to do so are available in EasyFSL. In this tutorial, we will show you how to use them.

## Extracting the features

EasyFSL has a `predict_embeddings()` method, which takes as input a DataLoader and a torch Module, and outputs a DataFrame with all your embeddings. Let's use it to extract all the embeddings from the test set of the CUB dataset. For a backbone, we are going to use the Swin Transformer pre-trained on ImageNet and directly available from torchvision. Note that we can do that because there is no intersection between CUB and ImageNet, so we are not technically cheating. Still, the resulting performance cannot be compared with that of a model trained on CUB's train set, since the training data is not the same.

First do some necessary configuration (this is not the interesting part).

In [ ]:
try:
    import google.colab

    colab = True
except:
    colab = False

if colab is True:
    # Running in Google Colab
    # Clone the repo
    !git clone https: // github.com / sicara / easy-few-shot-learning
    % cd easy-few-shot-learning
    !pip install.
else:
    # Run locally
    # Ensure working directory is the project's root
    # Make sure easyfsl is installed!
    %cd ..

Then we prepare the data and the model.

In [ ]:
# Download CUB if necessary
!make download-cub

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models
from easyfsl.datasets import CUB

batch_size = 128
num_workers = 1

# Ensure that you're working from the root of the repository
# and that CUB's images are in ./data/CUB/images/
dataset = CUB(split="test", training=False)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False,
)

model = torchvision.models.swin_v2_t(
    weights=torchvision.models.Swin_V2_T_Weights.IMAGENET1K_V1,
)
# Remove the classification head: we want embeddings, not ImageNet predictions
model.head = nn.Flatten()

# If you have a GPU, use it!
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

And now we extract the embeddings. This gives us a DataFrame with the embeddings of all the images in the test set, along with their respective class_names.

In [ ]:
from easyfsl.utils import predict_embeddings

embeddings_df = predict_embeddings(dataloader, model, device=device)

print(embeddings_df)

We now have our embeddings ready to use! We will not use the backbone anymore.

## Performing inference on pre-extracted embeddings

To deliver the embeddings to our Few-Shot Classifier, we will need an appropriate DataLoader. We will use the `FeaturesDataset` class from EasyFSL. Since we have a DataFrame ready to use, we will use the handy `from_dataset()` initializer from `FeaturesDataset`, but you can also use `from_dict()` to initialize from a dictionary, or the built-in constructor to initialize it directly from labels and embeddings.

In [ ]:
from easyfsl.datasets import FeaturesDataset

features_dataset = FeaturesDataset.from_dataframe(embeddings_df)

print(features_dataset[0])

Then, like in all other few-shot tutorials, we are going to build a DataLoader that loads batches in the shape of few-shot tasks:

In [ ]:
from easyfsl.samplers import TaskSampler

task_sampler = TaskSampler(
    features_dataset,
    n_way=5,
    n_shot=5,
    n_query=10,
    n_tasks=100,
)
features_loader = DataLoader(
    features_dataset,
    batch_sampler=task_sampler,
    num_workers=num_workers,
    pin_memory=True,
    collate_fn=task_sampler.episodic_collate_fn,
)

We now need to instantiate our Few-Shot Classifier. We will use a Prototypical Network for simplicity, but you can use any other model from EasyFSL.

Since we are working directly on features, **we don't need to initialize Prototypical Networks with a backbone**.

In [ ]:
from easyfsl.methods import PrototypicalNetworks

# Default backbone if we don't specify anything is Identity.
# But we specify it anyway for clarity and robustness.
few_shot_classifier = PrototypicalNetworks(backbone=nn.Identity())

We can now evaluate our model on the test set, and just enjoy how fast it goes:

In [ ]:
from easyfsl.utils import evaluate

accuracy = evaluate(
    few_shot_classifier,
    features_loader,
    device="cpu",
)

print(f"Average accuracy : {(100 * accuracy):.2f} %")

And that is it! Notice that when you're working on pre-extracted embeddings, you can process tasks way faster (65 tasks/s on my MacBook Pro). This should always be your default settings whenever you're working with a method that uses a frozen backbone at test-time (that's most of them).

## Conclusion
Thanks for following this tutorial. If you have any issue, please [raise one](https://github.com/sicara/easy-few-shot-learning/issues), and if EasyFSL is helping you, do not hesitate to [star the repository](https://github.com/sicara/easy-few-shot-learning).